In [1]:
cd ..

/Users/sgemma.sun/Documents/data101/airbnb-ml


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import src.transform as trans

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
import s3fs
import pyarrow.parquet as pq
import src.const as const

s3 = s3fs.S3FileSystem()

filePath = 's3://airbnb-barcelona/valid/currentDate=2020-03-13'
airbnb_df = pq.ParquetDataset(filePath, filesystem=s3).read_pandas().to_pandas()
airbnb_df = airbnb_df.drop(columns=const.drop_cat_columns)
airbnb = airbnb_df.drop(columns=const.drop_num_columns)

# Transformation

In [3]:
cat_df = airbnb.select_dtypes(include=['object']).copy()

NameError: name 'airbnb' is not defined

In [ ]:
# drop outliers
airbnb = trans.drop_rows_occurs_less_than(airbnb, "cancellation_policy", 2)
airbnb = trans.drop_rows_occurs_less_than(airbnb, "neighbourhood_group_cleansed", 1)
airbnb = trans.drop_rows_occurs_less_than(airbnb, "host_response_time", 1)

In [ ]:
# fillna
airbnb = airbnb.fillna(value={"host_is_superhost": False})
airbnb = airbnb.fillna(value={"host_has_profile_pic": False})
airbnb = airbnb.fillna(value={"host_identity_verified": False})
airbnb = airbnb.fillna(value={"host_response_time": "N/A"})
airbnb = trans.fillna_with_lowest_occurance(airbnb, "last_review")

In [ ]:
# boolean_to_float
airbnb = trans.foldleft(trans.encode_boolean_to_float, airbnb, const.boolean_to_float_cols)

In [ ]:
# element count
airbnb = trans.extract_num_of_items_for_column(airbnb, "host_verifications")
airbnb = trans.extract_num_of_items_for_column(airbnb, "amenities")

In [ ]:
# numeric value
airbnb = airbnb.fillna(airbnb.mean())

In [ ]:
# date columns
ymd_to_time = trans.string_to_timestamp('%Y-%m-%d')
airbnb["last_review"] = airbnb["last_review"].apply(ymd_to_time)
days_from_2020_03_11 = trans.days_from_date(compare_date=pd.to_datetime('2020-03-11', format='%Y-%m-%d'))
airbnb["last_review_days_fr_20200311"] = airbnb["last_review"].apply(days_from_2020_03_11)

# Visualization

In [12]:
airbnb.head(10)

,host_id,host_response_time,host_response_rate,host_is_superhost,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood_group_cleansed,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,price,security_deposit,cleaning_fee,guests_included,extra_people,has_availability,availability_365,number_of_reviews,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,last_review_days_fr_20200311
0,71615,within an hour,0.990000,0.0,5,1.0,1.0,Sant Martí,41.40889,2.18555,1.0,Apartment,Entire home/apt,6.0,1.0,2.0,4.0,Real Bed,18,130.0,150.000000,42.000000,2.0,25.0,1.0,304.0,1.0,2015-10-10,80.0,10.0,10.0,2.0,10.0,10.0,8.0,0.0,flexible,0.0,0.0,30.0,30.0,0.0,0.0,0.02,1614
1,71615,within an hour,0.990000,0.0,5,1.0,1.0,Eixample,41.40420,2.17306,1.0,Apartment,Entire home/apt,8.0,2.0,3.0,6.0,Real Bed,22,60.0,150.000000,50.000000,2.0,30.0,1.0,312.0,20.0,2019-10-19,87.0,9.0,9.0,10.0,10.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,30.0,30.0,0.0,0.0,0.25,144
2,90417,within an hour,1.000000,1.0,8,1.0,1.0,Sant Martí,41.41203,2.22114,0.0,Apartment,Entire home/apt,6.0,2.0,3.0,5.0,Real Bed,46,210.0,300.000000,80.000000,3.0,10.0,1.0,359.0,51.0,2019-09-29,95.0,10.0,10.0,10.0,10.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,1.0,2.0,2.0,0.0,0.0,0.48,164
3,108310,within an hour,1.000000,1.0,8,1.0,1.0,Gràcia,41.40145,2.15645,1.0,Apartment,Private room,2.0,1.0,1.0,1.0,Real Bed,17,32.0,166.427606,43.103105,1.0,25.0,1.0,46.0,268.0,2019-11-06,95.0,10.0,9.0,10.0,10.0,10.0,10.0,1.0,strict_14_with_grace_period,1.0,1.0,1.0,0.0,1.0,0.0,2.38,126
4,136853,within an hour,0.920000,0.0,8,1.0,0.0,Gràcia,41.40950,2.15938,1.0,Apartment,Entire home/apt,4.0,1.0,1.0,1.0,Real Bed,10,60.0,200.000000,58.000000,4.0,0.0,1.0,324.0,182.0,2019-10-16,92.0,9.0,9.0,8.0,9.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,39.0,39.0,0.0,0.0,1.71,147
5,136853,within an hour,0.920000,0.0,8,1.0,0.0,Gràcia,41.40928,2.16112,1.0,Apartment,Entire home/apt,5.0,1.5,3.0,3.0,Real Bed,10,70.0,200.000000,108.000000,5.0,0.0,1.0,326.0,90.0,2019-11-05,88.0,9.0,9.0,9.0,9.0,9.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,39.0,39.0,0.0,0.0,0.84,127
6,141199,N/A,0.941213,0.0,4,1.0,0.0,Les Corts,41.38720,2.14088,1.0,Apartment,Private room,1.0,1.0,1.0,1.0,Real Bed,25,30.0,0.000000,0.000000,1.0,15.0,1.0,144.0,19.0,2014-05-12,99.0,10.0,10.0,10.0,10.0,9.0,9.0,0.0,moderate,0.0,0.0,1.0,0.0,1.0,0.0,0.17,2130
7,135703,within an hour,1.000000,0.0,8,1.0,1.0,Gràcia,41.40464,2.16954,1.0,Apartment,Entire home/apt,6.0,1.5,2.0,3.0,Real Bed,36,140.0,200.000000,75.000000,4.0,20.0,1.0,350.0,59.0,2019-10-19,86.0,8.0,9.0,10.0,9.0,10.0,9.0,1.0,flexible,0.0,0.0,14.0,14.0,0.0,0.0,0.58,144
8,73163,within a few hours,1.000000,0.0,8,1.0,1.0,Ciutat Vella,41.37916,2.17535,1.0,Apartment,Private room,2.0,1.0,1.0,1.0,Real Bed,17,100.0,150.000000,40.000000,1.0,0.0,1.0,180.0,8.0,2013-07-15,68.0,8.0,8.0,7.0,9.0,8.0,7.0,0.0,moderate,0.0,0.0,2.0,1.0,1.0,0.0,0.07,2431
9,73163,within a few hours,1.000000,0.0,8,1.0,1.0,Ciutat Vella,41.37859,2.17730,1.0,Apartment,Entire home/apt,8.0,3.0,4.0,6.0,Real Bed,38,250.0,240.000000,78.000000,6.0,50.0,1.0,165.0,142.0,2019-10-08,90.0,10.0,9.0,10.0,10.0,10.0,9.0,1.0,strict_14_with_grace_period,0.0,0.0,2.0,1.0,1.0,0.0,1.28,155


# Encode category before training model

In [19]:
# category encode
category_encoder = trans.encode_category_dic(airbnb)
category_dic = trans.foldleft(category_encoder, {}, const.category_columns)
dic_host_response_time = {'host_response_time': {'N/A': 1, 'a few days or more': 2, 'within a day': 3, 'within a few hours': 4, 'within an hour': 5}}
category_dic = dict(dic_host_response_time, **category_dic)
airbnb = airbnb.replace(category_dic)

# Price prediction

In [9]:
y = airbnb["price"]
X = airbnb.drop(columns=['price'])

In [14]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
scaler.transform(X)

array([[ 0.6673665 ,  0.3464109 , -0.47385002, ...,  2.25404242,
         2.25404242,  2.25404242],
       [ 0.6673665 ,  0.3464109 , -0.47385002, ...,  2.25404242,
         2.25404242,  2.25404242],
       [ 0.6673665 ,  0.41741589,  2.11037238, ...,  2.20208554,
         2.20208554,  2.20208554],
       ...,
       [-0.04380398, -0.15062408, -0.47385002, ..., -1.38399938,
        -1.38399938, -1.38399938],
       [ 0.6673665 ,  0.41741589,  2.11037238, ..., -1.30553389,
        -1.30553389, -1.30553389],
       [-0.75497446, -1.49971904, -0.47385002, ..., -1.52184415,
        -1.52184415, -1.52184415]])

In [15]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
params = {}
model = LinearRegression(fit_intercept=False)
grid_search = GridSearchCV(model, params, cv=5, scoring=('r2'))
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=False,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None, param_grid={},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [17]:
best_model = grid_search.best_estimator_


In [1]:
from sklearn.metrics import r2_score

def compute_mse_r2 (_model, x_test, y_test):
    y_predict = _model.predict(x_test)
    r2 = r2_score(y_test, y_predict)
    mse = mean_squared_error(y_test, y_predict)
    return (r2, mse)